In [1]:
import sys
# sys.path.remove('/home/dquintero/venvs/dssat/lib/python3.10/site-packages')
sys.path.append("/home/dquintero/venvs/serviceDSSAT/lib/python3.10/site-packages")
sys.path.append('/home/dquintero/spatialDSSAT')
sys.path.append('/home/dquintero/dev_dssat_service/')
# sys.path.append('..')
from database import connect
from dssat import run_spatial_dssat
from datetime import datetime

In [2]:
dbname = "dssatserv"

In [4]:
# df = run_spatial_dssat(
#     dbname=dbname, 
#     schema="kenya", 
#     admin1="Nakuru",
#     plantingdate=datetime(2021,3, 15),
#     cultivar="990003",
#     nitrogen=[(0, 50), (30, 40)]
# )

In [16]:
import ipywidgets as widgets

In [6]:
con = connect(dbname)

In [7]:
cur = con.cursor()
query = "SELECT admin1 FROM kenya.admin"
cur.execute(query)
admin1_list = [i[0] for i in cur.fetchall()]
cur.close()

In [8]:
cultivar_types = {'Short': "990003", 'Medium': "990002", 'Long': "990001"}

In [9]:
def run(pars):
    df = run_spatial_dssat(
            dbname=dbname, 
            schema="kenya", 
            admin1=pars["admin1"],
            plantingdate=datetime(pars["plantingdate"].year, pars["plantingdate"].month, pars["plantingdate"].day),
            cultivar=cultivar_types[pars["cultivar"]],
            nitrogen=[(0, pars["nitrogen"]),]
        )
    return df

In [10]:
from highcharts_core.chart import Chart
from highcharts_core.chart import HighchartsOptions
from highcharts_core.options.series.boxplot import BoxPlotSeries
from highcharts_core.options.series.scatter import ScatterSeries
from highcharts_core.options.series.area import LineSeries

In [11]:
from matplotlib.cm import get_cmap
from matplotlib.colors import to_hex

colors = list(map(to_hex, get_cmap("tab10").colors))
len(colors)

10

In [12]:
import numpy as np
def boxplot_data(data):
    data = np.array(data)
    q1, median, q3 = np.quantile(data, [0.25, .5, .75])
    iqr = q3 - q1
    low = max(median - 1.5*iqr, min(data))
    high = min(median + 1.5*iqr, max(data))
    outliers = data[(data > high) | (data < low)]
    return {"low": low, "q1": q1, "median": median, "q3": q3, "high": high}, outliers

def add_box(chart, data):
    boxdata, outliers = boxplot_data(data)
    label = f"{pars['cultivar']}<br>{pars['plantingdate'].strftime('%b %d')}<br>{pars['nitrogen']}"
    if chart.options.series is None:
        x = 0
    else:
        x = len(chart.options.series)/2
    boxdata["x"] = x
    # Boxplot
    box = BoxPlotSeries()
    box.name = label
    box.id = label
    box.data = [boxdata]
    box.group_padding = 0.
    box.grouping = False
    box.color = colors[int(x)%len(colors)]
    chart.add_series(box)
    
    # Outliers
    outs = ScatterSeries.from_array(
        np.array([[x]*len(outliers), outliers]).T
    )
    outs.linked_to = label
    outs.color = colors[int(x)%len(colors)]
    outs.marker = {"symbol": "circle"}
    chart.add_series(outs)
    return box
    

In [17]:
def init_chart():
    my_chart = Chart()
    my_chart.options = HighchartsOptions()
    my_chart.options.title = {
        'text': 'DSSAT maize yield estimates', 
        "style": {
            "font-size": "15px"
        }
    }
    my_chart.options.y_axis = {
        "title": {
            'text': 'Yield (kg/ha)', 
            "style": {
                "font-size": "15px"
            }
        },
        "labels": {
            "style": {
                "font-size": "15px",
            }
        }
    }
    my_chart.options.x_axis = {
        "title": {
            'text': 'Experiment', 
            "style": {
                "font-size": "15px",
            }
        },
        "labels": {
            "style": {
                "font-size": "15px",
            }
        }
    }
    return my_chart
my_chart = init_chart()

In [18]:
import plotly.graph_objects as go
from IPython.display import display, clear_output

In [19]:
pars = {"plantingdate": datetime(2022, 1, 1), "cultivar": "Short", "nitrogen": 30., "admin1": admin1_list[0]}

def on_value_change(change, par):
    pars[par] = change.new

region_picker = widgets.Dropdown(
    options=admin1_list,
    value=pars["admin1"],
    description='Admin1:',
    disabled=False,
)
region_picker.observe(lambda x: on_value_change(x, "admin1"), names='value')

plantingdate_picker = widgets.DatePicker(
    description='Planting date:',
    disabled=False,
    value=pars["plantingdate"]
)
plantingdate_picker.observe(lambda x: on_value_change(x, "plantingdate"), names='value')

cultivar_picker = widgets.Dropdown(
    options=['Short', 'Medium', 'Long'],
    description='Cultivar type:',
    disabled=False,
    value=pars["cultivar"]
)
cultivar_picker.observe(lambda x: on_value_change(x, "cultivar"), names='value')

nitrogen_slider = widgets.FloatSlider(
    min=0,
    max=120.0,
    step=0.1,
    description='Nitrogen rate:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='.1f',
    value=pars["nitrogen"]
)
nitrogen_slider.observe(lambda x: on_value_change(x, "nitrogen"), names='value')

button = widgets.Button(
    description='Run DSSAT',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Run DSSAT',
)
clear_button = widgets.Button(
    description='Clear',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Clear',
)

output = widgets.Output()

select_box = widgets.VBox([
    widgets.HBox([
        widgets.HBox([button, clear_button]),
        widgets.VBox([
            widgets.HBox([region_picker, plantingdate_picker]),
            widgets.HBox([cultivar_picker, nitrogen_slider])
        ])
    ]), 
    output])
# fig = go.Figure()
# def clear_figure(b):
#     fig.layout = {}
#     fig.data = []
#     clear_output()
#     display(select_box, output)
#     fig.show()
chart = init_chart()
def clear_figure(b):
#     clear_output()
    chart.options.series = None
    with output:
#         chart = clean_chart
        chart.display()
# def on_clic(b):
#     df = run(pars)
#     label = f"{pars['cultivar']}<br>{pars['plantingdate'].strftime('%b %d')}<br>{pars['nitrogen']}"
#     with output:
#         fig.add_trace(go.Box(y=df.HARWT.astype(int), name=label))
# #         fig.add_trace(go.Bar(y=[df.HARWT.astype(int).mean()], name=label))
#         clear_output()
#         fig.show()
def on_clic(b):
    df = run(pars)
    with output:
        add_box(chart, df.HARWT.astype(float))
# #         fig.add_trace(go.Bar(y=[df.HARWT.astype(int).mean()], name=label))
        clear_output()
        chart.display()
        
clear_button.on_click(clear_figure)
button.on_click(on_clic)
display(select_box, output)

Output()

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 50/50 [00:03<00:00, 15.88it/s]


In [20]:
df = run(pars)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 50/50 [00:03<00:00, 16.18it/s]


['#8dd3c7',
 '#ffffb3',
 '#bebada',
 '#fb8072',
 '#80b1d3',
 '#fdb462',
 '#b3de69',
 '#fccde5',
 '#d9d9d9',
 '#bc80bd',
 '#ccebc5',
 '#ffed6f']

In [180]:
len(my_chart.options.series)

1

In [239]:
np.array([[0, 0], [3, 4]]).T

array([[0, 3],
       [0, 4]])